In [2]:
import datetime, time, math, os, json, shutil, asyncio, networkx as nx
from utils.gitutils import create_pull_request, clone_repo, create_branch, wipe_repo, prepare_repo
from utils.agent import Agent, GenerationConfig, Interaction, Team
from utils.stringutils import arr_from_sep_string, extract_markdown_blocks, markdown_to_dict, fuzzy_find, skwonk
from utils.filetreeutils import FileTree, write_file_tree
from utils.listutils import flatten
from utils.frameworkutils import DartAnalyzer
from dotenv import load_dotenv
from utils.graphutils import loose_level_order, collapsed_level_order
from utils.frameworkutils import Framework

In [3]:
load_dotenv()  # Load environment variables from .env

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
API_KEY = os.getenv('API_KEY')

In [4]:
repo_url = 'https://github.com/6165-MSET-CuttleFish/TeamTrack'
source = Framework.FLUTTER
feature = f"""# Name
Organizations
## Descriptions
Organizations are a way to group users together. They can be used to organize users into teams, or to provide a way to manage permissions across a group of users. An organization is also a way for users to quickly share events with the members of the organization quickly.
"""

In [5]:
# Clone the repo and make a new branch
repo = clone_repo(repo_url)
base_branch = repo.active_branch.name
created_branch = f"modification-{base_branch}"
local_repo_path = str(repo.working_dir)
working_dir_path = f'{local_repo_path}\\{source.get_working_dir()}'

In [6]:
source_swe_summarizer = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_analyzer",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to summarize the functionality of provided code files. Structure your response as follows:

# filename.ext:

$7 sentence summary of the functionality/contents of the file$
'''
)
pm = Agent(
    model_name="gemini-1.5-pro-001",
    api_key=API_KEY,
    name="pm",
    generation_config=GenerationConfig(temperature=0.3),
    system_prompt=f"""You are a high-level technical project manager tasked with the project of adding functionality to a {source} repository."""
)
source_swe_coder = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_coder",
    generation_config=GenerationConfig(temperature=0.5),
    system_prompt=f"You are a {source} software engineer. Your job is to write code to implement a specific feature. Respond with code and nothing else. No explanations needed."
)
source_swe_editor = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_editor",
    generation_config=GenerationConfig(temperature=0.0),
    system_prompt=f"""You are a {source} software engineer tasked with editing a large code file to implement changes in accordance with the instructions you will receive in the following prompts. Large edits at once are NOT the way. We will solve this problem by dividing the edit into smaller sub-edits consisting of additions and deletions (no more than 10 lines per edit). We will use a custom file type called 'skinnydiff', which are diff-like files but only feature a larger file's substring, to convey these changes. Here is an example of a 'skinnydiff' file:

```skinnydiff
 void foo() {{
- int x = 5;
+ var x = 7;
  return x + 3; // Example code 
 }}
```
Skinnydiffs are shorter versions of diff files that only include a few context lines around the deletions and insertions. They DO NOT CONTAINS @@...@@  as this is wasted information

Given this knowledge, structure your response as follows:

# Step 1
$Brief description of the 1st sub-change$
## Scope
$Brief planning on which line or class to include in the change$
## Change 1
```skinnydiff
$code diff$
```

# Step N
$Brief description of the Nth sub-change$
## Scope
$Brief planning on which line or class to include in the change$
## Change N
```skinnydiff
$...$
```"""
)

team = Team(source_swe_summarizer, pm, source_swe_coder, source_swe_editor)

In [7]:
analyzer = source.get_analyzer(working_dir_path)

source_dependency_graph = analyzer.buildDependencyGraph()

In [8]:
# loose level order
eval_order = loose_level_order(source_dependency_graph)[::-1]
for level in eval_order:
    print(level)
print(f"Number of levels: {len(eval_order)}")

['models\\AppModel.dart', 'models\\StatConfig.dart']
['models\\GameModel.dart']
['models\\ScoreModel.dart']
['components\\misc\\PlatformGraphics.dart', 'functions\\Extensions.dart']
['components\\statistics\\BarGraph.dart', 'components\\users\\PFP.dart']
['components\\users\\UsersRow.dart', 'functions\\Statistics.dart', 'providers\\Auth.dart', 'components\\scores\\ScoreSummary.dart', 'components\\scores\\Incrementor.dart']
['views\\home\\match\\MatchView.dart']
['functions\\Functions.dart', 'components\\statistics\\PercentChange.dart']
['components\\scores\\ScoreTimeline.dart', 'models\\Change.dart', 'components\\scores\\ScoreRangeSummary.dart']
['views\\home\\match\\ExampleMatchRow.dart', 'components\\misc\\EmptyList.dart', 'views\\home\\match\\MatchRow.dart', 'components\\misc\\CardView.dart', 'components\\scores\\ScoringElementStats.dart', 'components\\statistics\\CheckList.dart', 'views\\home\\change\\ChangeRow.dart', 'views\\home\\change\\ChangeConfig.dart', 'views\\home\\match\\M

In [9]:
(source_file_tree := FileTree.from_dir(working_dir_path))

├── api\
│   ├── README.MD
├── components\
│   ├── misc\
│   │   ├── CardView.dart
│   │   ├── Collapsible.dart
│   │   ├── EmptyList.dart
│   │   ├── InfoPills.dart
│   │   ├── PlatformGraphics.dart
│   ├── scores\
│   │   ├── Incrementor.dart
│   │   ├── ScoreCard.dart
│   │   ├── ScoreRangeSummary.dart
│   │   ├── ScoreSummary.dart
│   │   ├── ScoreTimeline.dart
│   │   ├── ScoringElementStats.dart
│   ├── statistics\
│   │   ├── BarGraph.dart
│   │   ├── CheckList.dart
│   │   ├── PercentChange.dart
│   ├── users\
│   │   ├── PFP.dart
│   │   ├── UsersRow.dart
│   ├── AutonomousDrawingTool.dart
├── functions\
│   ├── APIMethods.dart
│   ├── Extensions.dart
│   ├── Functions.dart
│   ├── GPTModel.dart
│   ├── ResponseModel.dart
│   ├── Statistics.dart
├── models\
│   ├── AppModel.dart
│   ├── Change.dart
│   ├── GameModel.dart
│   ├── GPTModel.dart
│   ├── ScoreModel.dart
│   ├── StatConfig.dart
├── providers\
│   ├── Auth.dart
│   ├── PushNotifications.dart
│   ├── Theme.dart
│   ├

In [10]:
# Summarize the original repo
from tqdm import tqdm

async def summarize_group(group):
    async def summarize(node):
        name = source_file_tree.nodes[node]['name']
        content = source_file_tree.nodes[node]['content']
        message = f"{name}\n```\n{content}\n```"
        return await team.async_chat_with_agent(
            agent_name='source_swe_analyzer',
            message=message,
            context_keys=[f"summary_{neighbor}" for neighbor in source_dependency_graph[node]],
            save_keys=[f"summary_{node}", "all"],
            prompt_title=f"Summary of {node}"
            )
    tasks = [summarize(node) for node in group]
    responses = await asyncio.gather(*tasks)
    for i, response in enumerate(responses):
        source_file_tree.nodes[group[i]]["summary"] = response

for level in tqdm(eval_order):
    await summarize_group(level)
    time.sleep(0.5)

  0%|          | 0/19 [00:00<?, ?it/s]

100%|██████████| 19/19 [01:07<00:00,  3.58s/it]


In [11]:
prompt = f"""This is the file tree for the original {source} repo:
```
{source.get_working_dir()}\\
{source_file_tree}
```
Given the prior context, summarize the functionality of the entire repository succinctly.
"""
print(team.chat_with_agent('source_swe_analyzer', prompt, context_keys=["all"], save_keys=["all"], prompt_title=f"""Repository Summary.
File Tree provided:
```
{source.get_working_dir()}\\
{source_file_tree}
```
"""))

This Flutter repository implements a comprehensive application designed for managing and analyzing data related to First Tech Challenge (FTC) robotics competitions. 

The application allows users to:

- **Manage events:** Create, share, edit, and delete events, including local, remote, and driver analysis events.
- **Track team performance:** View team details, scores, match history, and statistics.
- **Visualize autonomous routines:** Capture, store, and view autonomous paths for robots.
- **Simulate alliances:** Explore potential alliances and receive AI-powered recommendations based on team performance.
- **Manage user permissions:** Grant access to events and control user roles.
- **Import data from The Orange Alliance API:** Retrieve event information and match schedules.
- **Analyze match schedules from images:** Capture images of match schedules and extract team data using Google ML Kit.
- **Manage inbox notifications:** Receive event invitations, block users, and manage notific

In [12]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

20816

In [13]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

122951

In [14]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.8306967816447203

In [15]:
response_format = f"""
You may choose ACTIONS from the following list:
- | WRITE | $FILE_PATH$ | $PROMPT$ (PROMPT should be a brief instruction for how the new FILE_PATH should be WRITTEN)
- | EDIT | $FILE_PATH$ | $PROMPT$ (PROMPT should be a brief instruction for how the FILE_PATH should be EDITED)
- | DELETE | $FILE_PATH$ (Delete the provided file)
----------
Structure your response using the following example:
```
| WRITE | {source.get_working_dir()}\\path\\to\\newfile.ext | Implement ... with the following properties:
    - property1
    - property2
    - dependency1
    - dependency2
| EDIT | {source.get_working_dir()}\\path\\to\\file.ext | Fix the bug ... by doing:
    - step1
    - step2
| DELETE | {source.get_working_dir()}\\path\\to\\some\\file.ext | Reasons for deletion:
    - reason1
    - reason2
```
It is VERY important that you only respond with a high level overview of what to do. The actual implementation will be done by the coder.
"""

# Create new file tree
prompt = f'''
You are to build the following feature:
{feature}
------------------
{response_format}
------------------
You need to follow the response template structure exactly (MY JOB DEPENDS ON IT)
'''
print(response := team.chat_with_agent("pm", prompt, context_keys=["all"], save_keys=["all"], prompt_title="Feature Creation Steps"))

```
| WRITE | lib\models\Organization.dart | Implement an `Organization` class with the following properties:
    - `id`: String
    - `name`: String
    - `members`: List<String> (User IDs)
    - `admins`: List<String> (User IDs)
| WRITE | lib\providers\OrganizationProvider.dart | Implement an `OrganizationProvider` class that uses Firestore to:
    - Create new organizations.
    - Get an organization by ID.
    - Get a list of organizations a user is a member of.
    - Add and remove members from an organization.
    - Update organization details.
| WRITE | lib\views\organizations\OrganizationSettingsScreen.dart | Implement an `OrganizationSettingsScreen` widget to:
    - Display organization details (name, members, admins).
    - Allow admins to edit the organization name.
    - Allow admins to promote members to admins.
    - Allow admins to remove members from the organization.
| WRITE | lib\views\organizations\CreateOrganizationScreen.dart | Implement a `CreateOrganizationScreen

In [16]:
md = extract_markdown_blocks(response)[0]
blocks = md.split("|")
actions = []
i = 0
while i < len(blocks):
    block = blocks[i].strip()
    if block != "WRITE" and block != "EDIT" and block != "DELETE":
        i += 1
        continue
    action, file, prompt = blocks[i:i + 3]
    action = action.strip()
    if file.strip().startswith(source.get_working_dir()):
        file = file.strip()[len(source.get_working_dir()) + 1:]
    else:
        file = file.strip()
    prompt = prompt.strip()
    actions.append((action, file, prompt))
    i += 3
for action in actions:
    print(action)

('WRITE', 'models\\Organization.dart', 'Implement an `Organization` class with the following properties:\n    - `id`: String\n    - `name`: String\n    - `members`: List<String> (User IDs)\n    - `admins`: List<String> (User IDs)')
('WRITE', 'providers\\OrganizationProvider.dart', 'Implement an `OrganizationProvider` class that uses Firestore to:\n    - Create new organizations.\n    - Get an organization by ID.\n    - Get a list of organizations a user is a member of.\n    - Add and remove members from an organization.\n    - Update organization details.')
('WRITE', 'views\\organizations\\OrganizationSettingsScreen.dart', 'Implement an `OrganizationSettingsScreen` widget to:\n    - Display organization details (name, members, admins).\n    - Allow admins to edit the organization name.\n    - Allow admins to promote members to admins.\n    - Allow admins to remove members from the organization.')
('WRITE', 'views\\organizations\\CreateOrganizationScreen.dart', 'Implement a `CreateOrgan

In [17]:
source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens

21231

In [18]:
# Cache the context threads
factor = 32768 / source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens
# eg. if factor is 3.2, round to 4
factor = math.ceil(factor)
thread = team.context_threads['all'].copy()
thread *= factor
source_swe_coder.model.count_tokens('\n'.join(message['parts'][0] for interaction in thread for message in interaction.to_dict())).total_tokens

42431

In [24]:
def write(file_path, prompt):
    response = source_swe_coder.chat(f"Instructions for {file_path}:\n{prompt}")
    md = extract_markdown_blocks(response)[0]
    path = f"{working_dir_path}\\{file_path}"
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, 'w') as f:
        f.write(md)

def edit(file_path, prompt):
    with open(f"{working_dir_path}\\{file_path}", 'r') as f:
        content = f.read()
    response = source_swe_editor.chat(
        f"{prompt}\n\nRemember, keep your skinnydiffs SHORT and do not include @@...@@. Instead, split it into multiple steps.",
        custom_context=[Interaction(f"Contents of {file_path}", content)]
    )
    diffs = extract_markdown_blocks(response)
    for diff in diffs:
        with open(f'{working_dir_path}\\{file_path}', 'w') as f:
            f.seek(0)
            f.truncate()
            content = skwonk(content, diff)
            f.write(content)

def delete(file_path):
    os.remove(f"{working_dir_path}\\{file_path}")

In [25]:
from utils.errorutils import repeat_until_finish
from tqdm import tqdm

for i in tqdm(range(5, len(actions))):
    action, file, prompt = actions[i]
    if action == 'WRITE':
        repeat_until_finish(lambda: write(file, prompt), max_retries=2)
    elif action == 'EDIT':
        repeat_until_finish(lambda: edit(file, prompt), max_retries=2)
    elif action == 'DELETE':
        delete(file)

 50%|█████     | 1/2 [00:01<00:01,  1.65s/it]

No similar substring found in database
["Row(\n                   children: [\n                     IconButton(\n                       onPressed: _onPressed,\n                       tooltip: 'Select Contact',\n                       icon: Icon(Icons.contacts),\n                       color: Theme.of(context).colorScheme.primary,\n                     ),\n                     IconButton(\n                       onPressed: _onPressedOrg,\n                       tooltip: 'Select Organization',\n                       icon: Icon(Icons.business),\n                       color: Theme.of(context).colorScheme.primary,\n                     ),\n                     Expanded(\n                       child: PlatformTextField(\n                         textInputAction: TextInputAction.done,\n                         placeholder: 'Email',\n                         keyboardType: TextInputType.emailAddress,\n                         controller: emailController,\n                         autoCorrect:

100%|██████████| 2/2 [00:15<00:00,  7.59s/it]

["import 'package:flutter/material.dart';\nimport 'package:teamtrack/models/AppModel.dart';\nimport 'package:teamtrack/components/misc/PlatformGraphics.dart';\nimport 'package:provider/provider.dart';\n\nclass OrganizationSelection extends StatefulWidget {\n  const OrganizationSelection({Key? key}) : super(key: key);\n\n  @override\n  State<OrganizationSelection> createState() => _OrganizationSelectionState();\n}\n\nclass _OrganizationSelectionState extends State<OrganizationSelection> {\n  @override\n  Widget build(BuildContext context) {\n    final organizations = context.watch<TeamTrackUser>().organizations;\n    return Scaffold(\n      appBar: AppBar(\n        title: Text('Select Organization'),\n        backgroundColor: Theme.of(context).colorScheme.primary,\n      ),\n      body: ListView.builder(\n        itemCount: organizations.length,\n        itemBuilder: (context, index) {\n          final organization = organizations[index];\n          return ListTile(\n            title: 